In [1]:
import pandas as pd
import sqlite3
from datetime import datetime
import os
import re
import shutil
import logging

class VolSurfaceDatabase:
    """
    Vol Surface Database Manager for Credit Index Options
    """
    
    def __init__(self, db_path=None):
        """Initialize database connection and logging"""
        if db_path is None:
            self.db_path = r"C:\source\repos\psc\packages\psc_csa_tools\credit_macro\data\raw\vol_surfaces.db"
        else:
            self.db_path = db_path
        
        # Ensure directory exists
        os.makedirs(os.path.dirname(self.db_path), exist_ok=True)
        
        # Setup logging
        log_path = os.path.join(os.path.dirname(self.db_path), "vol_surface_updates.log")
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_path),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def check_if_date_exists(self, data_date):
        """Check if data for specific date already exists"""
        if not os.path.exists(self.db_path):
            return False
            
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Check if table exists first
        cursor.execute("""
            SELECT name FROM sqlite_master 
            WHERE type='table' AND name='vol_surfaces'
        """)
        
        if not cursor.fetchone():
            conn.close()
            return False
        
        cursor.execute('''
            SELECT COUNT(*) FROM vol_surfaces 
            WHERE data_date = ?
        ''', (data_date,))
        
        count = cursor.fetchone()[0]
        conn.close()
        
        return count > 0
    
    def create_or_update_database(self, excel_path, data_date=None, force_update=False):
        """
        Create or update the vol surface database from Excel
        """
        if not os.path.exists(excel_path):
            self.logger.error(f"Excel file not found: {excel_path}")
            return 0
        
        if data_date is None:
            data_date = datetime.now().strftime("%Y-%m-%d")
        
        # Check if data already exists
        if not force_update and self.check_if_date_exists(data_date):
            self.logger.info(f"Data for {data_date} already exists. Use force_update=True to overwrite.")
            return 0
        
        self.logger.info(f"Processing vol surfaces for date: {data_date}")
        
        # Read Excel file
        xl_file = pd.ExcelFile(excel_path)
        sheet_names = xl_file.sheet_names
        self.logger.info(f"Found {len(sheet_names)} sheets")
        
        # Connect to database
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Create tables if they don't exist
        self._create_tables(cursor)
        
        # Delete existing data for this date if force_update
        if force_update and self.check_if_date_exists(data_date):
            cursor.execute('DELETE FROM vol_surfaces WHERE data_date = ?', (data_date,))
            cursor.execute('DELETE FROM surface_metadata WHERE data_date = ?', (data_date,))
            self.logger.info(f"Deleted existing data for {data_date}")
        
        # Process each sheet
        total_options = 0
        for sheet_name in sheet_names:
            count = self._process_sheet(cursor, xl_file, sheet_name, data_date)
            total_options += count
        
        conn.commit()
        conn.close()
        
        self.logger.info(f"Total options stored: {total_options}")
        
        return total_options
    
    def _create_tables(self, cursor):
        """Create database tables with proper schema"""
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS vol_surfaces (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                data_date DATE NOT NULL,
                sheet_name TEXT NOT NULL,
                index_name TEXT NOT NULL,
                tenor TEXT NOT NULL,
                expiry DATE NOT NULL,
                spot_level REAL,
                forward_level REAL NOT NULL,
                atm_strike REAL,
                strike REAL NOT NULL,
                option_type TEXT NOT NULL,
                bid REAL,
                ask REAL,
                mid REAL,
                delta REAL,
                vol REAL NOT NULL,
                change REAL,
                breakeven REAL,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                UNIQUE(data_date, index_name, tenor, strike, option_type)
            )
        ''')
        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS surface_metadata (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                data_date DATE NOT NULL,
                sheet_name TEXT NOT NULL,
                index_name TEXT NOT NULL,
                tenor TEXT NOT NULL,
                expiry DATE NOT NULL,
                spot_level REAL,
                forward_level REAL NOT NULL,
                atm_strike REAL,
                update_time TEXT,
                UNIQUE(data_date, sheet_name)
            )
        ''')
        
        # Create indices for faster queries
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_vol_date ON vol_surfaces(data_date)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_vol_index ON vol_surfaces(index_name, tenor)')
    
    def _process_sheet(self, cursor, xl_file, sheet_name, data_date):
        """Process a single sheet and store data"""
        
        # Extract tenor from sheet name
        tenor = self._extract_tenor(sheet_name)
        if not tenor:
            self.logger.warning(f"Skipping {sheet_name} - cannot extract tenor")
            return 0
        
        # Read sheet data
        df = xl_file.parse(sheet_name, header=None)
        
        # Parse the data
        metadata, options = self._parse_sheet_data(df, sheet_name, tenor)
        
        if not metadata or not options:
            self.logger.warning(f"No valid data found in {sheet_name}")
            return 0
        
        # Add data_date
        metadata['data_date'] = data_date
        
        # Store metadata
        cursor.execute('''
            INSERT OR REPLACE INTO surface_metadata 
            (data_date, sheet_name, index_name, tenor, expiry, 
             spot_level, forward_level, atm_strike, update_time)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            data_date,
            sheet_name,
            metadata['index_name'],
            tenor,
            metadata['expiry'],
            metadata.get('spot_level'),
            metadata['forward_level'],
            metadata['atm_strike'],
            metadata.get('update_time')
        ))
        
        # Store options
        for option in options:
            cursor.execute('''
                INSERT OR REPLACE INTO vol_surfaces 
                (data_date, sheet_name, index_name, tenor, expiry,
                 spot_level, forward_level, atm_strike, strike,
                 option_type, bid, ask, mid, delta, vol, change, breakeven)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                data_date,
                sheet_name,
                metadata['index_name'],
                tenor,
                metadata['expiry'],
                metadata.get('spot_level'),
                metadata['forward_level'],
                metadata['atm_strike'],
                option['strike'],
                option['option_type'],
                option.get('bid'),
                option.get('ask'),
                option.get('mid'),
                option.get('delta'),
                option['vol'],
                option.get('change'),
                option.get('breakeven')
            ))
        
        self.logger.info(f"  {sheet_name}: {len(options)} options stored")
        
        return len(options)
    
    def _extract_tenor(self, sheet_name):
        """Extract tenor from sheet name"""
        # Handle both '_' and '-' separators
        if '_' in sheet_name:
            parts = sheet_name.split('_')
        elif '-' in sheet_name:
            parts = sheet_name.split('-')
        else:
            return None
        
        if len(parts) > 1:
            tenor = parts[-1]
            if re.match(r'^\d+m$', tenor):
                return tenor
        
        return None
    
    def _parse_sheet_data(self, df, sheet_name, tenor):
        """Parse sheet data to extract metadata and options"""
        
        metadata = {'tenor': tenor}
        options = []
        
        # Convert to lines for parsing
        lines = []
        for _, row in df.iterrows():
            line = ' | '.join([str(cell) if pd.notna(cell) else '' for cell in row])
            if line.strip():
                lines.append(line)
        
        # Parse metadata from header lines
        for line in lines[:5]:
            # Update time
            if 'Last updated:' in line:
                match = re.search(r'(\d{1,2}-\w{3}-\d{4})', line)
                if match:
                    metadata['update_time'] = match.group(1)
            
            # Index, expiry, forward, ATM
            if 'Options:' in line:
                # Determine index type
                if 'MAIN' in line.upper():
                    metadata['index_name'] = 'EU_IG'
                elif 'XOVER' in line.upper() or 'XO' in line:
                    metadata['index_name'] = 'EU_XO'
                elif 'HY' in line:
                    metadata['index_name'] = 'US_HY'
                elif 'IG' in line:
                    metadata['index_name'] = 'US_IG'
                
                # Extract expiry date
                match = re.search(r'(\d{1,2}-\w{3}-\d{2})', line)
                if match:
                    metadata['expiry'] = match.group(1)
                
                # Extract forward level
                match = re.search(r'Fwd\s*@([\d.]+)', line)
                if match:
                    metadata['forward_level'] = float(match.group(1))
                
                # Extract ATM strike
                match = re.search(r'Delta\s*@([\d.]+)', line)
                if match:
                    metadata['atm_strike'] = float(match.group(1))
        
        # Validate metadata
        if 'index_name' not in metadata or 'forward_level' not in metadata:
            return None, None
        
        # Parse option data lines
        for line in lines:
            if '|' not in line or ('Delta' in line and 'Vol' in line):
                continue
            
            parts = line.split('|')
            
            # Process each option (Receiver and Payer pairs)
            for i in range(0, len(parts), 2):
                if i+1 >= len(parts):
                    break
                
                strike_part = parts[i].strip()
                data_part = parts[i+1].strip() if i+1 < len(parts) else ''
                
                # Extract strike
                strike_match = re.search(r'([\d.]+)', strike_part)
                if not strike_match or not data_part:
                    continue
                
                try:
                    strike = float(strike_match.group(1))
                    option_type = 'Receiver' if i == 0 else 'Payer'
                    
                    # Parse option details
                    option = self._parse_option_details(data_part, strike, option_type)
                    if option and option['vol']:
                        options.append(option)
                except:
                    continue
        
        return metadata, options
    
    def _parse_option_details(self, data_str, strike, option_type):
        """Parse individual option data"""
        
        tokens = data_str.split()
        if len(tokens) < 2:
            return None
        
        option = {
            'strike': strike,
            'option_type': option_type,
            'vol': None
        }
        
        # Bid/Ask
        if '/' in tokens[0]:
            try:
                parts = tokens[0].split('/')
                option['bid'] = float(parts[0])
                option['ask'] = float(parts[1])
                option['mid'] = (option['bid'] + option['ask']) / 2
            except:
                pass
        
        # Process remaining tokens
        for i, token in enumerate(tokens[1:], 1):
            if '%' in token:
                try:
                    option['delta'] = float(token.rstrip('%'))
                except:
                    pass
            elif i == 2 and re.match(r'^[\d.]+$', token):
                option['vol'] = float(token)
            elif i == 3 and re.match(r'^-?[\d.]+$', token):
                option['change'] = float(token)
            elif i == 4 and re.match(r'^[\d.]+$', token):
                option['breakeven'] = float(token)
        
        return option if option['vol'] else None
    
    def query_surface(self, index_name=None, tenor=None, data_date=None):
        """Query vol surface data"""
        
        conn = sqlite3.connect(self.db_path)
        
        query = "SELECT * FROM vol_surfaces WHERE 1=1"
        params = []
        
        if index_name:
            query += " AND index_name = ?"
            params.append(index_name)
        
        if tenor:
            query += " AND tenor = ?"
            params.append(tenor)
        
        if data_date:
            query += " AND data_date = ?"
            params.append(data_date)
        
        query += " ORDER BY strike, option_type"
        
        df = pd.read_sql_query(query, conn, params=params)
        conn.close()
        
        return df

# Daily update script (save as separate file: update_vol_surfaces.py)
def daily_update():
    """
    Daily update script - run via Windows Task Scheduler
    """
    vol_db = VolSurfaceDatabase()
    
    excel_path = r"C:\Users\alessandro.esposito\Portman Square Capital LLP\Portman Square Capital - Documents\S\CSA\Credit Index Trading\vol_db.xlsx"
    
    today = datetime.now().strftime("%Y-%m-%d")
    
    if not vol_db.check_if_date_exists(today):
        total = vol_db.create_or_update_database(excel_path, data_date=today)
        print(f"Updated {total} options for {today}")
    else:
        print(f"Data for {today} already exists")

# Main execution
if __name__ == "__main__":
    
    vol_db = VolSurfaceDatabase()
    
    excel_path = r"C:\Users\alessandro.esposito\Portman Square Capital LLP\Portman Square Capital - Documents\S\CSA\Credit Index Trading\vol_db.xlsx"
    
    today = datetime.now().strftime("%Y-%m-%d")
    
    if not vol_db.check_if_date_exists(today):
        total = vol_db.create_or_update_database(excel_path, data_date=today)
        print(f"Processed {total} options for {today}")
    else:
        print(f"Data for {today} already exists. Use force_update=True to overwrite.")
    
    # Windows Task Scheduler setup instructions
    print("\n" + "="*60)
    print("To set up automated daily updates:")
    print("1. Save the daily_update() function as 'update_vol_surfaces.py'")
    print("2. Open Windows Task Scheduler")
    print("3. Create Basic Task > Daily > Set time (e.g., 8:00 AM)")
    print("4. Action: Start a program")
    print("5. Program: python.exe")
    print("6. Arguments: update_vol_surfaces.py")
    print("7. Start in: [directory of the script]")

Data for 2025-09-29 already exists. Use force_update=True to overwrite.

To set up automated daily updates:
1. Save the daily_update() function as 'update_vol_surfaces.py'
2. Open Windows Task Scheduler
3. Create Basic Task > Daily > Set time (e.g., 8:00 AM)
4. Action: Start a program
5. Program: python.exe
6. Arguments: update_vol_surfaces.py
7. Start in: [directory of the script]


In [ ]:
import sqlite3
import pandas as pd
import shutil
import os

# Define paths
source_db = "vol_surfaces.db"
target_dir = r"C:\source\repos\psc\packages\psc_csa_tools\credit_macro\data\raw"
target_db = os.path.join(target_dir, "jpm_vol_surfaces.db")

# Create directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)

# Check if target database already exists
if os.path.exists(target_db):
    print(f"Database already exists at: {target_db}")
    use_existing = True
else:
    # Check if source database exists to copy
    if os.path.exists(source_db):
        shutil.copy2(source_db, target_db)
        print(f"Database copied to: {target_db}")
        use_existing = False
    else:
        print(f"No database found at source ({source_db}) or target ({target_db})")
        print("Please run the vol surface parser first to create the database")
        exit()

# Connect to the database and examine schema
if os.path.exists(target_db):
    conn = sqlite3.connect(target_db)
    cursor = conn.cursor()
    
    print("\n" + "="*60)
    print("DATABASE SCHEMA")
    print("="*60)
    
    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    
    if not tables:
        print("Database exists but contains no tables. Run the parser to populate it.")
        conn.close()
        exit()
    
    for table in tables:
        table_name = table[0]
        print(f"\nTable: {table_name}")
        print("-" * 40)
        
        # Get table structure
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns = cursor.fetchall()
        
        # Format column info
        for col in columns:
            col_id, name, dtype, notnull, default, pk = col
            pk_text = " [PRIMARY KEY]" if pk else ""
            null_text = " NOT NULL" if notnull else ""
            default_text = f" DEFAULT {default}" if default else ""
            print(f"  {name:20} {dtype:10}{pk_text}{null_text}{default_text}")
    
    print("\n" + "="*60)
    print("DATABASE CONTENTS SUMMARY")
    print("="*60)
    
    # Count records in each table
    for table in tables:
        table_name = table[0]
        cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
        count = cursor.fetchone()[0]
        print(f"{table_name}: {count} records")
    
    # Check if vol_surfaces table exists and has data
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='vol_surfaces'")
    if cursor.fetchone():
        cursor.execute("SELECT COUNT(*) FROM vol_surfaces")
        count = cursor.fetchone()[0]
        
        if count > 0:
            # Show sample data structure
            print("\n" + "="*60)
            print("VOL_SURFACES TABLE - DATA STRUCTURE")
            print("="*60)
            
            sample_query = """
            SELECT 
                sheet_name,
                index_name,
                tenor,
                expiry,
                forward_level,
                strike,
                option_type,
                bid,
                ask,
                delta,
                vol
            FROM vol_surfaces
            LIMIT 5
            """
            
            df_sample = pd.read_sql_query(sample_query, conn)
            print(df_sample.to_string())
            
            # Show unique values for key fields
            print("\n" + "="*60)
            print("UNIQUE VALUES IN KEY FIELDS")
            print("="*60)
            
            # Unique indices
            cursor.execute("SELECT DISTINCT index_name FROM vol_surfaces ORDER BY index_name")
            indices = [row[0] for row in cursor.fetchall()]
            print(f"\nIndices: {indices}")
            
            # Unique expiries
            cursor.execute("SELECT DISTINCT expiry FROM vol_surfaces ORDER BY expiry")
            expiries = [row[0] for row in cursor.fetchall()]
            print(f"\nExpiries: {expiries}")
            
            # Unique tenors
            cursor.execute("SELECT DISTINCT tenor FROM vol_surfaces ORDER BY tenor")
            tenors = [row[0] for row in cursor.fetchall()]
            print(f"\nTenors: {tenors}")
            
            # Check for data dates if column exists
            cursor.execute("PRAGMA table_info(vol_surfaces)")
            columns = [col[1] for col in cursor.fetchall()]
            if 'data_date' in columns:
                cursor.execute("SELECT DISTINCT data_date FROM vol_surfaces ORDER BY data_date DESC")
                dates = [row[0] for row in cursor.fetchall()]
                print(f"\nData dates: {dates[:5]}")  # Show first 5 dates
            
            # Strike ranges by index
            print("\n" + "="*60)
            print("STRIKE RANGES BY INDEX")
            print("="*60)
            
            strike_ranges = pd.read_sql_query("""
                SELECT 
                    index_name,
                    MIN(strike) as min_strike,
                    MAX(strike) as max_strike,
                    COUNT(DISTINCT strike) as num_strikes
                FROM vol_surfaces
                GROUP BY index_name
                ORDER BY index_name
            """, conn)
            
            print(strike_ranges.to_string(index=False))
        else:
            print("\nvol_surfaces table exists but is empty. Run the parser to populate it.")
    else:
        print("\nvol_surfaces table does not exist. Run the parser to create it.")
    
    conn.close()
    
    print(f"\n✓ Database is ready at: {target_db}")
else:
    print(f"Failed to create or access database at: {target_db}")

No database found at source (vol_surfaces.db) or target (C:\source\repos\psc\packages\psc_csa_tools\credit_macro\data\raw\jpm_vol_surfaces.db)
Please run the vol surface parser first to create the database
Failed to create or access database at: C:\source\repos\psc\packages\psc_csa_tools\credit_macro\data\raw\jpm_vol_surfaces.db


: 